# Case Study Details

We have gathered booking data for 2019 and 2023 in this table. These are not full years, but
Q1/Q2/Q3 for both years.

## Objective

The main goals of the assessment are to find what has changed and why and provide
hypotheses/recommendations for future growth- 
-  Investigate what happened with the metric “EPS” (Earn Per Seat). Was there any change in EPS between these two years, and if it is, why?
-  Based on the data, try to make at least 3 more hypotheses/recommendations about increasing sales/revenue and ranking them by business value.

Calculating eps

```SQL

SELECT 
    EXTRACT(YEAR FROM paidon_date) AS Year,
    SUM(total_usd) AS total_usd,
    SUM(seats) AS total_seats,
    SUM(total_usd) / SUM(seats) AS eps
FROM 
    analytic_test_booking
WHERE
    EXTRACT(YEAR FROM paidon_date) IS NOT NULL
GROUP BY 
    EXTRACT(YEAR FROM paidon_date)
ORDER BY 
    Year;

```

| Year | total_usd      | total_seats | eps                 |
|------|----------------|-------------|---------------------|
| 2019 | 16857805.94386296 | 987923      | 17.063886501137194  |
| 2023 | 40853409.6423527  | 3139188     | 13.014005418711049  |


The EPS for 2023 is lower than that of 2019. Let's analyze the reasons why.

To understand which variable caused the drop in eps from 2019 to 2023, we can compare the growth rates of total_usd and seats between these years. If the sum of total_usd grew at a slower rate than the sum of seats, this would lead to a decrease in eps. Let's validate this assumption.

```sql

WITH YearlyData AS (
    SELECT 
        EXTRACT(YEAR FROM paidon_date) AS Year,
        SUM(total_usd) AS t_total_usd,
        SUM(seats) AS total_seats
    FROM 
        analytic_test_booking
    WHERE
    EXTRACT(YEAR FROM paidon_date) IS NOT NULL
    GROUP BY 
        EXTRACT(YEAR FROM paidon_date)
),
GrowthRates AS (
    SELECT 
        Year,
        t_total_usd,
        total_seats,
        LAG(t_total_usd) OVER (ORDER BY Year) AS prev_total_usd,
        LAG(total_seats) OVER (ORDER BY Year) AS prev_total_seats
    FROM 
        YearlyData
)
SELECT 
    Year,
    t_total_usd,
    total_seats,
    (t_total_usd - prev_total_usd) / prev_total_usd * 100 AS total_usd_growth_rate,
    (total_seats - prev_total_seats) / prev_total_seats * 100 AS total_seats_growth_rate,
    t_total_usd / total_seats AS eps
FROM 
    GrowthRates
ORDER BY 
    Year;
```

| Year | t_total_usd       | total_seats | total_usd_growth_rate | total_seats_growth_rate | eps                 |
|------|-------------------|-------------|-----------------------|-------------------------|---------------------|
| 2019 | 16857805.94386296 | 987923      | NULL                  | NULL                    | 17.063886501137194  |
| 2023 | 40853409.6423527  | 3139188     | 142.34120251707654    | 217.7563                | 13.014005418711049  |


Hypothesis:

The observation that the growth in total seats is higher than the growth in total earnings (`total_usd`) can be attributed to several potential reasons. Here are some possible explanations:

1. **Price Reductions or Discounts:**
   - If there were price reductions, promotions, or discounts offered during 2023, this could lead to an increase in the number of seats sold while the total earnings did not increase proportionally.

2. **Change in Product/Service Mix:**
   - There might have been a shift in the mix of products or services being sold. For example, more lower-priced options could have been sold in 2023 compared to higher-priced options in 2019.

3. **Market Saturation:**
   - The market might be reaching a saturation point where additional seats sold do not correspond to proportional increases in revenue due to market competition or reduced willingness to pay.

4. **Economic Factors:**
   - Economic downturns or other macroeconomic factors could result in customers being more price-sensitive, leading to higher sales volumes at lower price points.

5. **Operational Changes:**
   - Operational changes, such as increasing capacity or adding more seats to meet demand without a corresponding increase in pricing, could lead to more seats being sold at the same or lower prices.

6. **Increased Competition:**
   - Increased competition may force the company to lower prices to remain competitive, resulting in higher seat sales but lower revenue per seat.

7. **Promotional Strategies:**
   - Aggressive marketing campaigns focusing on increasing volume rather than revenue could have driven up the number of seats sold without significantly increasing earnings.

8. **Customer Behavior Changes:**
   - Changes in customer behavior, such as a preference for cheaper options or shorter durations, could lead to an increase in the number of seats sold but not a proportional increase in total revenue.

To gain a clearer understanding of the specific reasons for your case, consider the following additional analyses:

- **Price Analysis:**
  - Analyze the average price per seat over the years to see if there has been a significant drop in prices.
  
- **Revenue Segmentation:**
  - Break down the total earnings by product or service categories to identify if certain segments have experienced a decline in average revenue per seat.

- **Promotional Impact:**
  - Examine the impact of promotions, discounts, and other pricing strategies over time.

- **Customer Demographics and Preferences:**
  - Study changes in customer demographics and preferences to understand if there is a shift towards cheaper or different types of services.

By conducting these additional analyses, you can pinpoint more accurately which factors are contributing to the observed trends in earnings and seat demand.

- **Price Analysis:**
  - Analyze the average price per seat over the years to see if there has been a significant drop in prices.

```sql
SELECT 
    year,
    quarter,
    AVG(eps) AS avg_eps
FROM (
    SELECT
        EXTRACT(YEAR FROM paidon_date) AS year,
        EXTRACT(QUARTER FROM paidon_date) AS quarter,
        total_usd / seats AS eps
    FROM 
        analytic_test_booking
    WHERE
        EXTRACT(YEAR FROM paidon_date) IS NOT NULL
) AS subquery
WHERE
    quarter IN (1, 2, 3)
GROUP BY 
    year,
    quarter
ORDER BY 
    year,
    quarter;
```

| year | quarter | avg_eps |
| ---- | ------- | ------- |
| 2019 | 1       | 17.24   |
| 2019 | 2       | 16.91   |
| 2019 | 3       | 17.90   |
| 2023 | 1       | 13.80   |
| 2023 | 2       | 12.95   |
| 2023 | 3       | 12.55   |


```sql
SELECT 
    YEAR(paidon_date) AS year,
    QUARTER(paidon_date) AS quarter,
    ROUND(AVG(netprice_usd), 2) AS average_netprice_usd
FROM 
    analytic_test_booking
WHERE 
    YEAR(paidon_date) IN (2019, 2023) 
    AND QUARTER(paidon_date) IN (1, 2, 3)
    AND YEAR(paidon_date) IS NOT NULL
GROUP BY 
    YEAR(paidon_date), 
    QUARTER(paidon_date)
ORDER BY 
    YEAR(paidon_date), 
    QUARTER(paidon_date);


```


| year | quarter | average_netprice_usd |
| ---- | ------- | -------------------- |
| 2019 | 1       | 14.00                |
| 2019 | 2       | 13.71                |
| 2019 | 3       | 14.52                |
| 2023 | 1       | 11.45                |
| 2023 | 2       | 10.72                |
| 2023 | 3       | 10.32                |

- **Revenue Segmentation:**
  - Break down the total earnings by product or service categories to identify if certain segments have experienced a decline in average revenue per seat.

``` SQL
WITH avg_eps AS (
    SELECT 
        class_id,
        ROUND(AVG(CASE WHEN year = 2019 THEN eps END), 2) AS avg_eps_2019,
        ROUND(AVG(CASE WHEN year = 2023 THEN eps END), 2) AS avg_eps_2023
    FROM (
        SELECT
            EXTRACT(YEAR FROM paidon_date) AS year,
            class_id,
            total_usd / seats AS eps
        FROM 
            analytic_test_booking
        WHERE
            EXTRACT(YEAR FROM paidon_date) IS NOT NULL
    ) AS subquery
    WHERE
        year IN (2019, 2023)
    GROUP BY 
        class_id
)
SELECT 
    class_id,
    avg_eps_2019,
    avg_eps_2023,
    ROUND(ABS(avg_eps_2019 - avg_eps_2023), 2) AS diff_avg_eps
FROM 
    avg_eps
WHERE 
    avg_eps_2019 IS NOT NULL 
    AND avg_eps_2023 IS NOT NULL
ORDER BY 
    diff_avg_eps DESC
LIMIT 10;

```

| class_id | avg_eps_2019 | avg_eps_2023 | diff_avg_eps |
| -------- | ------------ | ------------ | ------------ |
| 1918     | 2178.63      | 13.09        | 2165.54      |
| 1662     | 97.84        | 7.75         | 90.09        |
| 20       | 18.08        | 83.42        | 65.34        |
| 1597     | 78.25        | 25.62        | 52.63        |
| 28       | 21.94        | 57.11        | 35.17        |
| 1320     | 89.44        | 54.59        | 34.85        |
| 2728     | 59.86        | 26.31        | 33.55        |
| 34       | 6.77         | 36.16        | 29.39        |
| 1607     | 5.40         | 29.92        | 24.52        |
| 1155     | 43.35        | 18.84        | 24.51        |


- **Customer Demographics and Preferences:**
  - Study changes in customer demographics and preferences to understand if there is a shift towards cheaper or different types of services.

```sql
WITH avg_eps AS (
    SELECT 
        country_id,
        ROUND(AVG(CASE WHEN year = 2019 THEN eps END), 2) AS avg_eps_2019,
        ROUND(AVG(CASE WHEN year = 2023 THEN eps END), 2) AS avg_eps_2023
    FROM (
        SELECT
            EXTRACT(YEAR FROM paidon_date) AS year,
            country_id,
            total_usd / seats AS eps
        FROM 
            analytic_test_booking
        WHERE
            EXTRACT(YEAR FROM paidon_date) IS NOT NULL
    ) AS subquery
    WHERE
        year IN (2019, 2023)
    GROUP BY 
        country_id
)
SELECT 
    country_id,
    avg_eps_2019,
    avg_eps_2023,
    ROUND((avg_eps_2019 - avg_eps_2023), 2) AS diff_avg_eps
FROM 
    avg_eps
WHERE 
    avg_eps_2019 IS NOT NULL 
    AND avg_eps_2023 IS NOT NULL
ORDER BY 
    diff_avg_eps DESC
LIMIT 10;

```

| country_id | avg_eps_2019 | avg_eps_2023 | diff_avg_eps |
| ---------- | ------------ | ------------ | ------------ |
| AZ         | 57.27        | 11.83        | 45.44        |
| LR         | 46.08        | 8.00         | 38.08        |
| ZM         | 56.47        | 23.24        | 33.23        |
| ME         | 39.41        | 12.26        | 27.15        |
| PY         | 34.93        | 12.18        | 22.75        |
| GT         | 37.07        | 16.14        | 20.93        |
| CV         | 33.22        | 13.18        | 20.04        |
| CR         | 35.28        | 16.57        | 18.71        |
| SV         | 28.64        | 10.76        | 17.88        |
| CK         | 27.80        | 10.27        | 17.53        |


Based on the provided data schema and the identified drop in Eran per Seat (eps) from 2019 to 2023, here are three hypotheses and recommendations for increasing sales and revenue, ranked by business value:

### 1. **Optimize Pricing Strategy (High Business Value)**
   **Hypothesis:** Adjusting the pricing strategy to reflect market demand and competitive positioning can enhance revenue per seat.
   
   **Recommendations:**
   - **Dynamic Pricing:** Implement dynamic pricing models that adjust ticket prices based on factors such as demand, time to departure, and competition. For instance, increase prices during peak travel times and offer discounts during off-peak times.
   - **Segmentation:** Offer different pricing tiers based on customer segments, such as economy, business, and premium seats, to cater to varying customer preferences and willingness to pay.
   - **Promotions and Discounts:** Run targeted promotions and discounts for specific routes or times that typically experience lower occupancy to increase seat utiland overall business performance.

```sql
SELECT 
    month,
    total_seats,
    ROUND((total_seats / total_seats_sum) * 100, 2) AS percentage_of_total_seats
FROM (
    SELECT 
        DATE_FORMAT(paidon_date, '%Y-%m') AS month,
        SUM(seats) AS total_seats
    FROM 
        analytic_test_booking
    WHERE 
        YEAR(paidon_date) = 2023
        AND MONTH(paidon_date) BETWEEN 1 AND 9
    GROUP BY 
        DATE_FORMAT(paidon_date, '%Y-%m')
) AS monthly_seats
JOIN (
    SELECT 
        SUM(seats) AS total_seats_sum
    FROM 
        analytic_test_booking
    WHERE 
        YEAR(paidon_date) = 2023
        AND MONTH(paidon_date) BETWEEN 1 AND 9
) AS total_seats_info
ON 1=1
ORDER BY 
    total_seats DESC;
    
    ```
    

| month    | total_seats | percentage_of_total_seats |
| -------- | ----------- | ------------------------ |
| 2023-03  | 393919      | 12.55                    |
| 2023-08  | 367654      | 11.71                    |
| 2023-04  | 363668      | 11.58                    |
| 2023-07  | 360831      | 11.49                    |
| 2023-02  | 359459      | 11.45                    |
| 2023-01  | 355358      | 11.32                    |
| 2023-09  | 330456      | 10.53                    |
| 2023-05  | 321813      | 10.25                    |
| 2023-06  | 285993      | 9.11                     |


``` sql
SELECT 
    DAYNAME(paidon_date) AS day_of_week,
    SUM(seats) AS total_seats,
    ROUND(SUM(seats) * 100.0 / (SELECT SUM(seats) FROM analytic_test_booking WHERE YEAR(paidon_date) = 2023 AND MONTH(paidon_date) BETWEEN 1 AND 9), 2) AS percentage_of_total_seats
FROM 
    analytic_test_booking
WHERE 
    YEAR(paidon_date) = 2023
    AND MONTH(paidon_date) BETWEEN 1 AND 9
GROUP BY 
    DAYNAME(paidon_date)
ORDER BY 
    total_seats DESC;
```

| day_of_week | total_seats | percentage_of_total_seats |
| ----------- | ----------- | ------------------------ |
| Monday      | 465775      | 14.84                    |
| Sunday      | 459947      | 14.65                    |
| Wednesday   | 452813      | 14.42                    |
| Thursday    | 451270      | 14.38                    |
| Tuesday     | 450957      | 14.37                    |
| Friday      | 433300      | 13.80                    |
| Saturday    | 425089      | 13.54                    |


| hour_of_day | total_seats | percentage_of_total_seats |
| ----------- | ----------- | ------------------------ |
| 21:00       | 194896      | 6.21                     |
| 20:00       | 193091      | 6.15                     |
| 18:00       | 192144      | 6.12                     |
| 19:00       | 191879      | 6.11                     |
| 17:00       | 189104      | 6.02                     |
| 16:00       | 176834      | 5.63                     |
| 22:00       | 176319      | 5.62                     |
| 15:00       | 164419      | 5.24                     |
| 12:00       | 163178      | 5.20                     |
| 11:00       | 163087      | 5.20                     |


```sql
SELECT 
     -- country_id,
    -- seller_id,
     class_id,
     -- cust_id,
     -- user_agent,
     -- createdby,
     -- landing,
    round(SUM(total_usd), 2) AS total_usd,
    round ((SUM(total_usd) / (SELECT SUM(total_usd) FROM analytic_test_booking where YEAR(paidon_date) = 2023) * 100), 2) AS percentage_of_total
FROM 
    analytic_test_booking
where Year(paidon_date) = 2023
GROUP BY 
    -- country_id
    -- seller_id
      class_id
     -- cust_id
     -- user_agent,
     -- createdby
     -- landing
ORDER BY 
    total_usd DESC
LIMIT 10;

```

| class_id | total_usd    | percentage_of_total |
| -------- | ------------ | ------------------- |
| 43       | 5538807.74   | 13.56               |
| 21       | 5117352.51   | 12.53               |
| 12       | 2359512.50   | 5.78                |
| 6        | 1927556.58   | 4.72                |
| 11       | 1423242.72   | 3.48                |
| 79       | 1259985.51   | 3.08                |
| 44       | 1161400.66   | 2.84                |
| 5        | 874532.66    | 2.14                |
| 1776     | 684805.19    | 1.68                |
| 3473     | 630651.15    | 1.54                |


| country_id | total_usd    | percentage_of_total |
| ---------- | ------------ | ------------------- |
| TH         | 12816936.12  | 31.37               |
| VN         | 3371532.33   | 8.25                |
| ID         | 2355387.43   | 5.77                |
| SG         | 2042309.39   | 5.00                |
| FR         | 1744735.92   | 4.27                |
| PH         | 1695213.63   | 4.15                |
| DE         | 1680660.19   | 4.11                |
| GB         | 1613003.69   | 3.95                |
| NL         | 1436204.64   | 3.52                |
| US         | 1327477.10   | 3.25                |


The same code will be used to find relations with other variables

| seller_id | total_usd    | percentage_of_total |
| --------- | ------------ | ------------------- |
| 2547      | 3573473.40   | 8.75                |
| 16        | 2450293.31   | 6.00                |
| 191       | 1738954.53   | 4.26                |
| 2810      | 1458379.19   | 3.57                |
| 2356      | 1326626.45   | 3.25                |
| 269       | 1224183.98   | 3.00                |
| 181       | 1164656.65   | 2.85                |
| 2382      | 1117873.07   | 2.74                |
| 527       | 1014406.70   | 2.48                |
| 2548      | 961400.48    | 2.35                |


| user_agent                                                                                                                           | total_usd    | percentage_of_total |
| ------------------------------------------------------------------------------------------------------------------------------------- | ------------ | ------------------- |
| Mozilla/5.0 (iPhone; CPU iPhone OS 16_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Mobile/15E148 Safari/604.1 | 1704113.63   | 4.17                |
| Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Mobile/15E148 Safari/604.1 | 1634761.94   | 4.00                |
| Mozilla/5.0 (iPhone; CPU iPhone OS 16_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.2 Mobile/15E148 Safari/604.1 | 1409454.39   | 3.45                |
| Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 | 1073609.31   | 2.63                |
| api/booking                                                                                                                            | 986779.28    | 2.42                |
| Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1 | 866719.39    | 2.12                |
| Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Mobile Safari/537.36 | 811358.62    | 1.99                |
| Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 | 803600.83    | 1.97                |
| Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6.1 Mobile/15E148 Safari/604.1 | 777711.98    | 1.90                |
| Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 | 703656.87    | 1.72                |


| createdby | total_usd    | percentage_of_total |
| --------- | ------------ | ------------------- |
| 1         | 13102713.04  | 32.07               |
| 4         | 12961968.52  | 31.73               |
| 30        | 3535039.23   | 8.65                |
| 3481      | 1363260.13   | 3.34                |
| 6795      | 1125172.30   | 2.75                |
| 4087482   | 867932.77    | 2.12                |
| 64932     | 620355.82    | 1.52                |
| 56330     | 567092.21    | 1.39                |
| 20092     | 453332.18    | 1.11                |
| 26        | 452194.42    | 1.11                |


| landing                                   | total_usd    | percentage_of_total |
| ----------------------------------------- | ------------ | ------------------- |
| https://12go.asia/                       | 2678038.17   | 6.56                |
| https://www.12go.asia/en                  | 2600910.67   | 6.37                |
| https://12go.asia/en                      | 1850652.39   | 4.53                |
| https://12go.co/                         | 1206299.24   | 2.95                |
| https://www.12go.co/en                    | 667756.03    | 1.63                |
| https://12go.asia/fr                      | 333007.60    | 0.82                |
| https://secure.12go.asia/en/login         | 330881.29    | 0.81                |
| https://12go.asia/de                      | 327467.61    | 0.80                |
| https://www.12go.asia/de                  | 320521.65    | 0.78                |
| https://www.12go.asia/fr                  | 280009.97    | 0.69                |


### 2. **Enhance Marketing and Customer Acquisition Strategies (Medium to High Business Value)**
   **Hypothesis:** Improved marketing and targeted customer acquisition strategies can increase overall sales volume and revenue.
   
   **Recommendations:**
   - **Targeted Advertising:** Utilize data analytics to identify high-value customer segments and create targeted advertising campaigns through digital marketing channels, including social media, search engines, and email marketing.
   - **Loyalty Programs:** Develop and promote loyalty programs to encourage repeat bookings. Offer points or discounts for frequent travelers, which can increase customer retention and lifetime value.
   - **Partnerships:** Partner with travel agencies, corporate clients, and other relevant businesses to offer bundled deals and increase sales through their networks.

### 3. **Improve Customer Experience and Service Offerings (Medium Business Value)**
   **Hypothesis:** Enhancing the overall customer experience and expanding service offerings can lead to increased customer satisfaction, repeat business, and higher revenue per seat.
   
   **Recommendations:**
   - **Customer Feedback:** Regularly gather and analyze customer feedback to identify areas for improvement. Implement changes based on this feedback to enhance the overall travel experience.
   - **Additional Services:** Introduce value-added services such as in-transit entertainment, Wi-Fi, meal options, and comfortable seating arrangements. Offer these as premium services to increase revenue per seat.
   - **Ease of Booking:** Simplify the booking process on the website and mobile apps. Ensure a user-friendly interface and offer multiple payment options to cater to a broader audience.

### Data-Driven Approach:
To validate these hypotheses and implement the recommendations effectively, consider the following steps:

1. **Data Analysis:** Conduct detailed data analysis to understand customer behavior, booking patterns, and price sensitivity. Use historical data from 2019 and 2023 to identify trends and insights.
2. **A/B Testing:** Implement A/B testing for dynamic pricing models, marketing campaigns, and service enhancements to measure their impact on sales and revenue.
3. **Customer Surveys:** Perform surveys and gather qualitative data to supplement quantitative analysis. This can provide deeper insights into customer preferences and satisfaction levels.

By implementing these hypotheses and recommendations, the company can potentially increase its sales and revenue, thereby improving the Eran per Seat (eps) and overall business performance.

In [3]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [5]:
import mysql.connector
import pandas as pd
from mysql.connector import Error

# Define the SQL query to fetch the relevant data from the `analytic_test_booking` table
query = """
SELECT 
    netprice_usd,
    total_usd,
    seats
FROM 
    analytic_test_booking
"""

try:
    # Establish the connection
    connection = mysql.connector.connect(
        host='hrtest.12go.asia',
        port=20062,
        user='hrtest-ro',
        password='q2FTQezFKRmHp',
        database='12go'
    )

    if connection.is_connected():
        print("Connected to the database")

        # Create a cursor object using cursor() method
        cursor = connection.cursor()

        # Execute a sample query to confirm connection
        cursor.execute("SELECT DATABASE();")
        record = cursor.fetchone()
        print("You're connected to the database:", record)

        # Execute the main query
        cursor.execute(query)
        
        # Fetch all the results
        results = cursor.fetchall()

        # Create a DataFrame from the results
        df = pd.DataFrame(results, columns=['netprice_usd', 'total_usd', 'seats'])

        # Display the first few rows of the DataFrame
        print(df.head())

        # Drop rows with missing values for correlation analysis
        df = df.dropna()

        # Calculate correlation matrix
        correlation_matrix = df[['netprice_usd', 'total_usd', 'seats']].corr()

        # Print the correlation matrix
        print("Correlation matrix:")
        print(correlation_matrix)

        # Compute the correlation between specific pairs
        correlation_netprice_total_usd = df['netprice_usd'].corr(df['total_usd'])
        correlation_netprice_seats = df['netprice_usd'].corr(df['seats'])

        print(f"Correlation between netprice_usd and total_usd: {correlation_netprice_total_usd:.2f}")
        print(f"Correlation between netprice_usd and seats: {correlation_netprice_seats:.2f}")

except Error as e:
    print(f"An error occurred: {e}")

finally:
    # Ensure the connection is closed
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to the database
You're connected to the database: ('12go',)
   netprice_usd  total_usd  seats
0         17.81      48.98      2
1         17.81      48.98      2
2         20.25      48.34      2
3         34.12      74.91      2
4         12.24      15.58      1
Correlation matrix:
              netprice_usd  total_usd     seats
netprice_usd      1.000000   0.998185 -0.002073
total_usd         0.998185   1.000000  0.025414
seats            -0.002073   0.025414  1.000000
Correlation between netprice_usd and total_usd: 1.00
Correlation between netprice_usd and seats: -0.00
MySQL connection is closed
